In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.measure import label, regionprops
from scipy.ndimage import binary_dilation

In [ ]:
plt.rc('font', family = 'serif', size = 13, weight = "bold")
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.rc('axes', linewidth = 2)
plt.rc('lines', linewidth = 3)
plt.rc('figure', figsize = (5, 5), dpi = 80)
plt.rc('legend', fontsize = 13)

cm = plt.cm.get_cmap('tab20')

### Load in the image

In [ ]:
sender_image = plt.imread("./Sample_Data/Txn_series1_0min_sender.tif")
receiver_image = plt.imread("./Sample_Data/Txn_series1_0min_receiver.tif")

In [ ]:
fig = plt.figure(figsize=(11, 5), dpi = 80)

plt.subplot(1, 2, 1)
plt.title("Sender Channel (Raw)")
plt.imshow(sender_image, cmap="gray")

plt.subplot(1, 2, 2)
plt.title("Receiver Channel (Raw)")
plt.imshow(receiver_image, cmap="gray")

plt.tight_layout()
plt.show()

### Sender receiver droplet identification

In [ ]:
def droplet_detection(sender_image, receiver_image):
    """
    Identify the locations of the sender and receiver droplets.
    """
    # Create Binary for better signal to noise
    binary = sender_image > (sender_image.mean() + sender_image.std())

    # use labeling function to identify connected blobs
    label_im = label(binary)
    blob_lists = regionprops(label_im)

    # Remove small blobs
    senders = [i for i in blob_lists if i.area > 500]
    
    # Mask to document all sender locations
    mask = np.zeros_like(binary)
    for i in senders:
        minr, minc, maxr, maxc = i.bbox
        mask[minr:maxr, minc:maxc] = 1
        
    # Create Binary for better signal to noise
    binary = receiver_image > (receiver_image.mean() + receiver_image.std())
    
    # use labeling function to identify connected blobs
    label_im = label(binary)
    blob_lists = regionprops(label_im)

    # Remove small blobs
    receivers = [i for i in blob_lists if i.area > 500]

    # Extract receiver droplets
    true_receivers = []
    for i in receivers:
        minr, minc, maxr, maxc = i.bbox
        if mask[int((minr+maxr)/2), int((minc+maxc)/2)] == 0:
            true_receivers.append(i)
            
    return senders, true_receivers

In [ ]:
senders, receivers = droplet_detection(sender_image, receiver_image)

### Single-channel droplet identification

In [ ]:
def droplet_detection_single_channel(image):
    """
    Identify the locations of the droplets.
    """
    # Create Binary for better signal to noise
    binary = image > (image.mean() + image.std())

    # use labeling function to identify connected blobs
    label_im = label(binary)
    blob_lists = regionprops(label_im)

    # Remove small blobs
    droplets = [i for i in blob_lists if i.area > 500]
    
    return droplets

In [ ]:
droplets = droplet_detection_single_channel(sender_image)

### Extract the pixel values of droplets - Filter Method

In [ ]:
def droplet_value_extraction_v2(droplets, image, filename = False, dilation = False):
    """
    Extract the intensity value of the droplets. Can save as a txt file for later plotting.
    """
    
    values = []
    
    if dilation:
        
        for blob in droplets:

            minr, minc, maxr, maxc = blob.bbox
            blob_img = image[minr:maxr, minc:maxc]
            vals = blob_img[binary_dilation(blob.image)]
            values.append(vals)

        values = np.concatenate(values)
        
    else:
    
        for blob in droplets:

            minr, minc, maxr, maxc = blob.bbox
            blob_img = image[minr:maxr, minc:maxc]
            vals = blob_img[blob.image]
            values.append(vals)

        values = np.concatenate(values)
    
    if filename: np.savetxt(filename, values, fmt='%f')
    
    return values

In [ ]:
sender_values = droplet_value_extraction_v2(senders, receiver_image)

print("Mean:\t{:.2f}\nStd:\t{:.2f}\nMin:\t{:.2f}\nMax:\t{:.2f}".format(np.mean(sender_values), np.std(sender_values), np.min(sender_values), np.max(sender_values)))


In [ ]:
figure = plt.figure(figsize = (8, 6), dpi = 80)

plt.hist(sender_values, bins = 1000, color = 'grey')
plt.title("Sender Droplets", weight = "bold")
plt.xlabel("Fluorescence Intensity (a.u.)", weight = "bold")
plt.ylabel("Frequency", weight = "bold")
plt.xlim(0, 20000)
plt.show()

In [ ]:
receiver_values = droplet_value_extraction_v2(receivers, receiver_image, dilation = True)

print("Mean:\t{:.2f}\nStd:\t{:.2f}\nMin:\t{:.2f}\nMax:\t{:.2f}".format(np.mean(receiver_values), np.std(receiver_values), np.min(receiver_values), np.max(receiver_values)))


In [ ]:
figure = plt.figure(figsize = (8, 6), dpi = 80)

plt.hist(receiver_values, bins = 1000, color = 'grey')
plt.title("Receiver Droplets", weight = "bold")
plt.xlabel("Fluorescence Intensity (a.u.)", weight = "bold")
plt.ylabel("Frequency", weight = "bold")
plt.xlim(0, 20000)
plt.show()

### Generate droplet-annotated images

In [ ]:
def droplet_annotation(image, droplet, RAW = False, filename = False):
    """
    Saves the image with droplets annotated.
    
    RAW = True -> plots the raw image.
    
    RAW = False -> plots the amplified binary image.
    
    Set up a filename to save the image.
    """
    
    figure = plt.figure(figsize=(8, 8), dpi = 80)

    ax = plt.subplot(1,1,1)
    
    if RAW:
        plt.imshow(image, cmap = "gray")
    else:
        binary = image > (image.mean() + image.std())
        plt.imshow(binary, cmap = "gray")
        

    for i in droplet:
        minr, minc, maxr, maxc = i.bbox
        rect = plt.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2, alpha=.8)
        ax.add_patch(rect)
        ax.set_axis_off()
        
    if filename:
        plt.savefig(filename, bbox_inches='tight', pad_inches=0)

    plt.show()

In [ ]:
print("Sender Droplets")
droplet_annotation(receiver_image, senders, RAW = False, filename = "./Sample_Data/test.png")

In [ ]:
print("Receiver Droplets")
droplet_annotation(receiver_image, receivers, RAW = False)

## Time Series

In [ ]:
series_sender_values, series_receiver_values = [], []

for i in [0, 30, 60]:
    sender_image = plt.imread("./Sample_Data/Txn_series1_{:.0f}min_sender.tif".format(i))
    receiver_image = plt.imread("./Sample_Data/Txn_series1_{:.0f}min_receiver.tif".format(i))
    
    senders, receivers = droplet_detection(sender_image, receiver_image)
    sender_value = droplet_value_extraction_v2(senders, receiver_image)
    receiver_value = droplet_value_extraction_v2(receivers, receiver_image, dilation = True)
    series_sender_values.append(sender_value)
    series_receiver_values.append(receiver_value)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 3.5))

bplot1 = ax1.boxplot(series_sender_values,
                   labels = ["0", "30", "60"],
                   showfliers=False, # dont show outliers
                   vert=True,  # vertical box alignment
                   patch_artist=True)

bplot2 = ax2.boxplot(series_receiver_values,
                   labels = ["0", "30", "60"],
                   showfliers=False, # dont show outliers
                   vert=True,  # vertical box alignment
                   patch_artist=True)

ax1.yaxis.grid(True)
ax1.set_title("Senders", weight = "bold")
ax1.set_xlabel("Time (min)", weight = "bold")
ax1.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")

ax2.yaxis.grid(True)
ax2.set_title("Receivers", weight = "bold")
ax2.set_xlabel("Time (min)", weight = "bold")
ax2.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")


for bplot in (bplot1, bplot2):
    for patch, color in zip(bplot['boxes'], ["lightgray", "lightgray", "lightgray"]):
        patch.set_facecolor(color)  
    for median in bplot['medians']:
        median.set_color('black')
        
plt.tight_layout()
    
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 3.5))

ax1.hist(series_sender_values[0], bins = 1000, color = 'grey')
ax1.hist(series_sender_values[2], bins = 1000, color = 'lightgrey', alpha = .8)

ax1.set_title("Senders", weight = "bold")
ax1.set_xlabel("Fluorescence Intensity (a.u.)", weight = "bold")
ax1.set_ylabel("Frequency", weight = "bold")
ax1.set_xticklabels(np.arange(0, 20000, 2500), rotation=30)
ax1.set_xlim(0, 20000)

ax2.hist(series_receiver_values[0], bins = 1000, color = 'grey', label = "Before\nTranscription")
ax2.hist(series_receiver_values[2], bins = 1000, color = 'lightgrey', alpha = .8, label = "After\nTranscription\n(60min)")

ax2.set_title("Receivers", weight = "bold")
ax2.set_xlabel("Fluorescence Intensity (a.u.)", weight = "bold")
ax2.set_ylabel("Frequency", weight = "bold")
ax2.set_xticklabels(np.arange(0, 20000, 2500), rotation=30)
ax2.set_xlim(0, 20000)

plt.legend(loc=(1.05, 0.3))
      
plt.tight_layout()
    
plt.show()

### Intensity Distribution based on droplet sizes

In [ ]:
sender_image = plt.imread("./Sample_Data/Txn_series1_60min_sender.tif")
receiver_image = plt.imread("./Sample_Data/Txn_series1_60min_receiver.tif")

senders, receivers = droplet_detection(sender_image, receiver_image)

areas, diamters, values = [], [], []

for blob in receivers:
    # get the size(area) of the blob
    areas.append(blob.area)
    diamters.append(2*np.sqrt(blob.area/np.pi)*319.45/2048)
    # get the diameter
    minr, minc, maxr, maxc = blob.bbox
    blob_img = receiver_image[minr:maxr, minc:maxc]
    vals = blob_img[blob.image]
    values.append(vals)
    
receivers_df = pd.DataFrame({"area (pixel)":areas, 'diamter (um)':diamters, 'values':values})
receivers_df["means"] = receivers_df["values"].apply(np.mean)

areas, diamters, values = [], [], []

for blob in senders:
    # get the size(area) of the blob
    areas.append(blob.area)
    diamters.append(2*np.sqrt(blob.area/np.pi)*319.45/2048)
    # get the diameter
    minr, minc, maxr, maxc = blob.bbox
    blob_img = receiver_image[minr:maxr, minc:maxc]
    vals = blob_img[blob.image]
    values.append(vals)
    
senders_df = pd.DataFrame({"area (pixel)":areas, 'diamter (um)':diamters, 'values':values})
senders_df["means"] = senders_df["values"].apply(np.mean)

In [ ]:
receivers_df.head()

In [ ]:
senders_df.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 3.5))

ax1.scatter(senders_df['diamter (um)'], senders_df['means'], color = "grey", alpha = .8)

ax2.scatter(receivers_df['diamter (um)'], receivers_df['means'], color = "grey", alpha = .8)

ax1.set_title("Senders", weight = "bold")
ax1.set_xlabel("Diameter ($\mu$m)", weight = "bold")
ax1.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")

ax2.set_title("Receivers", weight = "bold")
ax2.set_xlabel("Diameter ($\mu$m)", weight = "bold")
ax2.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")
        
plt.tight_layout()
    
plt.show()

In [ ]:
def adv_concat(lst):
    try:
        lst = np.concatenate(lst)
    except:
        lst = np.array(lst)
    return lst

d5, d7_5, d10, d20, d30, d40, d50 = [], [], [], [], [], [], []

for i in range(len(senders_df)):
    if senders_df.iloc[i, 1] <= 5:
        d5.append(senders_df.iloc[i, 2])
    elif senders_df.iloc[i, 1] <= 7.5:
        d7_5.append(senders_df.iloc[i, 2])
    elif senders_df.iloc[i, 1] <= 10:
        d10.append(senders_df.iloc[i, 2])
    elif senders_df.iloc[i, 1] <= 20:
        d20.append(senders_df.iloc[i, 2])
    elif senders_df.iloc[i, 1] <= 30:
        d30.append(senders_df.iloc[i, 2])
    elif senders_df.iloc[i, 1] <= 40:
        d40.append(senders_df.iloc[i, 2])
    else:
        d50.append(senders_df.iloc[i, 2])

d5 = adv_concat(d5)
d7_5 = adv_concat(d7_5)
d10 = adv_concat(d10)
d20 = adv_concat(d20)
d30 = adv_concat(d30)
d40 = adv_concat(d40)
d50 = adv_concat(d50)

senders_dist = [d5, d7_5, d10, d20, d30, d40, d50]

d5, d7_5, d10, d20, d30, d40, d50 = [], [], [], [], [], [], []

for i in range(len(receivers_df)):
    if receivers_df.iloc[i, 1] <= 5:
        d5.append(receivers_df.iloc[i, 2])
    elif receivers_df.iloc[i, 1] <= 7.5:
        d7_5.append(receivers_df.iloc[i, 2])
    elif receivers_df.iloc[i, 1] <= 10:
        d10.append(receivers_df.iloc[i, 2])
    elif receivers_df.iloc[i, 1] <= 20:
        d20.append(receivers_df.iloc[i, 2])
    elif receivers_df.iloc[i, 1] <= 30:
        d30.append(receivers_df.iloc[i, 2])
    elif receivers_df.iloc[i, 1] <= 40:
        d40.append(receivers_df.iloc[i, 2])
    else:
        d50.append(receivers_df.iloc[i, 2])

d5 = adv_concat(d5)
d7_5 = adv_concat(d7_5)
d10 = adv_concat(d10)
d20 = adv_concat(d20)
d30 = adv_concat(d30)
d40 = adv_concat(d40)
d50 = adv_concat(d50)

receivers_dist = [d5, d7_5, d10, d20, d30, d40, d50]

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(9, 3.5))

bplot1 = ax1.boxplot(senders_dist,
                   showfliers=False, # dont show outliers
                   vert=True,  # vertical box alignment
                   patch_artist=True)

bplot2 = ax2.boxplot(receivers_dist,
                   showfliers=False, # dont show outliers
                   vert=True,  # vertical box alignment
                   patch_artist=True)

ax1.yaxis.grid(True)
ax1.set_title("Senders", weight = "bold")
ax1.set_xlabel("Diameter ($\mu$m)", weight = "bold")
ax1.set_xticklabels(["0-5", "5-7.5", "7.5-10", "10-20", "20-30", "30-40", "40-50"], rotation = 30)
ax1.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")

ax2.yaxis.grid(True)
ax2.set_title("Receivers", weight = "bold")
ax2.set_xlabel("Diameter ($\mu$m)", weight = "bold")
ax2.set_xticklabels(["0-5", "5-7.5", "7.5-10", "10-20", "20-30", "30-40", "40-50"], rotation = 30)
ax2.set_ylabel("Fluorescence Intensity (a.u.)", weight = "bold")


for bplot in (bplot1, bplot2):
    for patch, color in zip(bplot['boxes'], ["lightgray", "lightgray", "lightgray", "lightgray", "lightgray", "lightgray", "lightgray"]):
        patch.set_facecolor(color)  
    for median in bplot['medians']:
        median.set_color('black')
        
plt.tight_layout()
    
plt.show()

---
### Appendix

### Extract the pixel values of droplets - Threshold Method

In [ ]:
def droplet_value_extraction(droplets, image, filename = False, threshold = .75):
    """
    Extract the intensity value of the droplets. Can save as a txt file for later plotting.
    """
    
    values = []
    
    for blob in droplets:
        
        minr, minc, maxr, maxc = blob.bbox
        blob_img = image[minr:maxr, minc:maxc]
        vals = blob_img.flatten()
        background = np.quantile(image.flatten(), threshold)
        
        # Remove the background values
        vals = vals[vals > background]
        values.append(vals)
        
    values = np.concatenate(values)
    
    if filename: np.savetxt(filename, values, fmt='%f')
    
    return values

In [ ]:
sender_values = droplet_value_extraction(senders, receiver_image)

print("Mean:\t{:.2f}\nStd:\t{:.2f}\nMin:\t{:.2f}\nMax:\t{:.2f}".format(np.mean(sender_values), np.std(sender_values), np.min(sender_values), np.max(sender_values)))

plt.hist(sender_values, bins = 1000)
plt.xlim(0, 20000)
plt.show()

In [ ]:
receiver_values = droplet_value_extraction(receivers, receiver_image,\
                                           threshold = .75)

print("Mean:\t{:.2f}\nStd:\t{:.2f}\nMin:\t{:.2f}\nMax:\t{:.2f}".format(np.mean(receiver_values), np.std(receiver_values), np.min(receiver_values), np.max(receiver_values)))


plt.hist(receiver_values, bins = 1000)
plt.xlim(0, 20000)
plt.show()
np.mean(receiver_values)

### Notes

In [ ]:
senders, receivers = droplet_detection(sender_image, receiver_image)

In [ ]:
droplet_annotation(receiver_image, senders, RAW = False)
droplet_annotation(receiver_image, receivers, RAW = False)

In [ ]:
droplet = receivers[59]
droplet.area

In [ ]:
minx, miny, maxx, maxy = droplet.bbox
minx, miny, maxx, maxy, (maxx-minx)*(maxy-miny)

In [ ]:
plt.imshow(receiver_image[minx:maxx, miny:maxy], cmap = 'gray')

In [ ]:
plt.imshow(droplet.image, cmap = 'gray')

In [ ]:
from scipy.ndimage.morphology import binary_dilation

plt.imshow(binary_dilation(droplet.image), cmap = 'gray')

In [ ]:
values = receiver_image[minx:maxx, miny:maxy][droplet.image]
np.min(values), np.max(values), np.mean(values), np.std(values)

In [ ]:
for droplet in senders:
    minx, miny, maxx, maxy = droplet.bbox
    values = receiver_image[minx:maxx, miny:maxy][droplet.image]
    print(np.min(values), np.max(values), np.mean(values), np.std(values))


In [ ]:
ctr = 0
for droplet in receivers:
    minx, miny, maxx, maxy = droplet.bbox
    values = receiver_image[minx:maxx, miny:maxy][droplet.image]
    print(ctr, droplet.area, np.min(values), np.max(values), np.mean(values), np.std(values))
    ctr+=1
